In [ ]:
%store -r
import sys,json, os
job_class = os.getenv("DKUBE_JOB_CLASS")
if not job_class:
    !{sys.executable} -m pip install kfp==1.4.0 kfp-server-api==1.2.0 --user >/dev/null

In [ ]:
MONITOR_NAME = image_exp_config['MONITOR_NAME']
INPUT_TRAIN_TYPE = image_exp_config['INPUT_TRAIN_TYPE']
DKUBEUSERNAME = image_exp_config['SERVING_DKUBE_USERNAME']
TOKEN = image_exp_config['SERVING_DKUBE_TOKEN']
DKUBE_URL = image_exp_config['SERVING_DKUBE_URL']
TRAINING_DATASET = image_exp_config['TRAINING_DATASET']
DKUBE_TRAINING_CODE_NAME = image_exp_config['DKUBE_TRAINING_CODE_NAME']

## Importing libraries

In [ ]:
import json
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

import os
import kfp
import string
import random
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

from dkube.sdk.api import DkubeApi

## Loading Components

In [ ]:
dkube_preprocessing_op = components.load_component_from_file("/mnt/dkube/pipeline/components/preprocess/component.yaml")
dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml") 

In [ ]:
api = DkubeApi(URL=DKUBE_URL, token=TOKEN)
client = kfp.Client(
    host=os.getenv("KF_PIPELINES_ENDPOINT"),
    existing_token=TOKEN,
    namespace=DKUBEUSERNAME)

## Pipeline Inputs

In [ ]:
input_training_dataset = TRAINING_DATASET
training_program = DKUBE_TRAINING_CODE_NAME
## Training stage inputs
input_dataset_mount = ['/data']
training_script = "python image_cloudevents/training.py"
model_name = MONITOR_NAME
output_model_mount = "/model"

## Pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
    name='xray-pipeline',
    description='xray-training-pl'
)
def xray_pipeline(token):    
    train       = dkube_training_op(container=json.dumps({"image": "ocdr/dkube-datascience-tf-cpu:v2.0.0-10"}),
                                    framework="tensorflow", version="2.0.0",
                                    program=str(training_program), 
                                    run_script=str(training_script),
                                    datasets=json.dumps([str(input_training_dataset)]), 
                                    outputs=json.dumps([str(model_name)]),
                                    input_dataset_mounts=json.dumps(input_dataset_mount),
                                    output_mounts=json.dumps([str(output_model_mount)]),
                                    auth_token=token)
    
    serving     = dkube_serving_op(model=train.outputs['artifact'], device='cpu',
                                    name=MONITOR_NAME,
                                    serving_image=json.dumps({"image": "ocdr/tensorflowserver:2.0.0"}),
                                    auth_token=token, min_replicas = '1',
                                    production="true").after(train)

## Fields used for Configuring the Monitor through the UI

In [ ]:
# Get the deployment ID
deployment_id = api.get_deployment_id(name=MONITOR_NAME)

# Set up font definitions for output
class style:
   RED = '\033[91m\033[1m'
   BOLD = '\033[1m'
   END = '\033[0m'

# Print the instructions for the required fields for the UI-Based setup
print()
print(f"{style.BOLD}Note: These values will be needed as inputs during the UI-Based configuration process{style.END}")
print(f"{style.RED}Deployment ID (for Prefix/Subpath) = {style.END}", deployment_id)

In [ ]:
if not deployment_id:
    client.create_run_from_pipeline_func(xray_pipeline, arguments={'token':TOKEN})
else:
    print("Deployment Already Existing, skipping create")